# Experiment 4: Dataset Scaling Test
### Evaluating HQNN Performance Across Increasingly Complex Datasets

In [4]:
# Setup and Imports
import sys
sys.path.append("..")

import torch
from torch import nn, optim
from qiskit.visualization import circuit_drawer
from hqnn_core.qnn_builder import create_qnn
from hqnn_core.classical_model import ClassicalNN
from hqnn_core.hqnn_model import HybridModel
from hqnn_core.train import train_model
from hqnn_core.evaluate import evaluate_model
from hqnn_core.data_utils import load_rssi_dataset
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
# Experiment Configs
num_qubits = 3
reps = 1
feature_map_type = "zz"
ansatz_type = "real"
batch_size = 1
num_epochs = 20
learning_rate = 0.01

scenario_to_number = {
    "scenario_one": "1",
    "scenario_two": "2",
    "scenario_three": "3"
}

scenario_sets = [
    ["scenario_one"],
    ["scenario_one", "scenario_two"],
    ["scenario_one", "scenario_two", "scenario_three"]
]

In [6]:
# Model Training
results = []

for set_idx, scenarios in enumerate(scenario_sets):
    print(f"\n📦 Testing Scenario Set {set_idx + 1}: {scenarios}")
    
    # Load data from all scenarios in this set
    train_loaders, test_loaders = [], []
    for sc in scenarios:
        for signal in ["bluetooth", "wifi", "zigbee"]:
            tl, tstl = load_rssi_dataset(
                scenario=sc,
                signal=signal,
                sc_num=scenario_to_number[sc],
                batch_size=batch_size,
                base_path="../data"
            )
            train_loaders.append(tl)
            test_loaders.append(tstl)

    # Combine all data
    combined_train_loader = torch.utils.data.DataLoader(
        torch.utils.data.ConcatDataset([dl.dataset for dl in train_loaders]),
        batch_size=batch_size, shuffle=True
    )
    combined_test_loader = torch.utils.data.DataLoader(
        torch.utils.data.ConcatDataset([dl.dataset for dl in test_loaders]),
        batch_size=batch_size, shuffle=False
    )

    # Create model
    qnn = create_qnn(num_qubits, reps, feature_map_type, ansatz_type)
    clnn = ClassicalNN(input_size=3, hidden_size=32, output_size=2)
    model = HybridModel(qnn, clnn)

    # Train
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_model(model, combined_train_loader, criterion, optimizer, num_epochs, verbose=True)

    # Evaluate
    rmse = evaluate_model(model, combined_test_loader)
    results.append({"Scenario_Combination": "+".join(scenarios), "RMSE": rmse})


📦 Testing Scenario Set 1: ['scenario_one']
Epoch 1/20 - Loss: 7.8190
Epoch 2/20 - Loss: 2.0354
Epoch 3/20 - Loss: 2.7609
Epoch 4/20 - Loss: 1.8903
Epoch 5/20 - Loss: 1.5242
Epoch 6/20 - Loss: 1.2292
Epoch 7/20 - Loss: 1.3749
Epoch 8/20 - Loss: 1.0663
Epoch 9/20 - Loss: 1.0186
Epoch 10/20 - Loss: 1.0262
Epoch 11/20 - Loss: 0.9456
Epoch 12/20 - Loss: 0.9019
Epoch 13/20 - Loss: 0.8760
Epoch 14/20 - Loss: 0.8075
Epoch 15/20 - Loss: 0.9317
Epoch 16/20 - Loss: 0.8015
Epoch 17/20 - Loss: 0.8080
Epoch 18/20 - Loss: 0.8072
Epoch 19/20 - Loss: 0.7858
Epoch 20/20 - Loss: 0.8121
✅ RMSE on test set: 1.3066

📦 Testing Scenario Set 2: ['scenario_one', 'scenario_two']
Epoch 1/20 - Loss: 15.1394
Epoch 2/20 - Loss: 1.8624
Epoch 3/20 - Loss: 1.6920
Epoch 4/20 - Loss: 1.4363
Epoch 5/20 - Loss: 1.3374
Epoch 6/20 - Loss: 1.3100
Epoch 7/20 - Loss: 1.2041
Epoch 8/20 - Loss: 1.1896
Epoch 9/20 - Loss: 1.1503
Epoch 10/20 - Loss: 1.1159
Epoch 11/20 - Loss: 1.1679
Epoch 12/20 - Loss: 1.0777
Epoch 13/20 - Loss: 1.

In [7]:
# Results
df_results = pd.DataFrame(results)
df_results = df_results.sort_values("RMSE")
df_results.reset_index(drop=True, inplace=True)
df_results

,Scenario_Combination,RMSE
0,scenario_one+scenario_two,1.216235
1,scenario_one,1.306562
2,scenario_one+scenario_two+scenario_three,1.974671
